In [24]:
from random import *
from csv import reader
from math import exp

# Load a CSV file
print("hello")
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    
    return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            if minmax[i][1] - minmax[i][0]!=0 :
                row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Convert a dataset into list of lists
def convert(dataset):
    dataset_split = list()
    dataset_copy = list(dataset)
    for i in range(len(dataset)):
        line = list()
        line.append(dataset_copy[i])
        dataset_split.append(line)
    return dataset_split
def convert(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

def evaluate_algorithm(dataset, algorithm, *args):
    splitted = convert(dataset,5)
    for line in splitted:
        train_set = list(splitted)
        train_set.remove(line)
        train_set = sum(train_set, [])
        for row in line:
            row_copy = list(row)
            row_copy[-1] = None
        network = algorithm(train_set, *args)
    return network

# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random()-0.5 for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random()-0.5 for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# Backpropagation Algorithm 
def back_propagation(train, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    return network
def normalize(data):
    for i in range(len(data[0])-1):
        str_column_to_float(data, i)
    str_column_to_int(data, len(data[0])-1)
    minmax = dataset_minmax(data)
    #normalize_dataset(data, minmax)
    
    
# load and prepare data
train_file = 'optdigitsTrain.csv'
test_file = 'optdigitsTest.csv'
train_data = load_csv(train_file)
test_data = load_csv(test_file)
normalize(train_data)
normalize(test_data)
l_rate = 0.3
n_epoch = 10
n_hidden = 10
network = evaluate_algorithm(train_data, back_propagation, l_rate, n_epoch, n_hidden)
n_inputs = len(train_data[0])-1
n_outputs = len(set([row[-1] for row in train_data]))

#network = initialize_network(n_inputs , 33 , n_outputs )
#check network on test set and gather data for confusion matrix 
rows, cols = (10,10) 
confusionMatrix = [[0] * cols for i in range(rows)]
equal=0
for row in test_data:
    actual=row[-1]
    prediction = predict(network, row)
    if actual==prediction:
        equal+=1
    confusionMatrix[actual][prediction]+=1
for i in range (len(confusionMatrix)):
    for j in range(len(confusionMatrix)):
        print("actual: %i , predicted: %i , frequency: %.3f%%" %(i,j,100*confusionMatrix[i][j]/float(len(test_data))))
print('Accuracy: %.3f%%' % (100*(equal/float(len(test_data)))))
print(confusionMatrix)

hello
actual: 0 , predicted: 0 , frequency: 1.948%
actual: 0 , predicted: 1 , frequency: 0.000%
actual: 0 , predicted: 2 , frequency: 0.835%
actual: 0 , predicted: 3 , frequency: 0.111%
actual: 0 , predicted: 4 , frequency: 0.000%
actual: 0 , predicted: 5 , frequency: 0.835%
actual: 0 , predicted: 6 , frequency: 0.000%
actual: 0 , predicted: 7 , frequency: 1.447%
actual: 0 , predicted: 8 , frequency: 0.056%
actual: 0 , predicted: 9 , frequency: 4.897%
actual: 1 , predicted: 0 , frequency: 0.000%
actual: 1 , predicted: 1 , frequency: 7.513%
actual: 1 , predicted: 2 , frequency: 2.282%
actual: 1 , predicted: 3 , frequency: 0.000%
actual: 1 , predicted: 4 , frequency: 0.000%
actual: 1 , predicted: 5 , frequency: 0.056%
actual: 1 , predicted: 6 , frequency: 0.000%
actual: 1 , predicted: 7 , frequency: 0.111%
actual: 1 , predicted: 8 , frequency: 0.000%
actual: 1 , predicted: 9 , frequency: 0.223%
actual: 2 , predicted: 0 , frequency: 0.000%
actual: 2 , predicted: 1 , frequency: 0.000%
actu